Las herramientas que el agente puede ocupar incluyen servicios de búsqueda en internet que permiten extender el conocimiento no disponible por la LLM ya que añadimos esos hechos al contexto de la consulta. Pero veremos que estos esquemas pueden extenderse también con mecanismos de búsqueda como lo son las bases de datos vectoriales.

Gracias a estos medio podemos realizar un amplia recolección de información previo a su envío a la LLM para obtener el resultado final.

In [1]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

No es tarea trivial ya que encontrar información pertinente para resolver nuestra consulta requiere de encontrar la información relevante de nuestra base de conocimiento. Esto puede realizarse de distintas formas, mediante palabras clave o metadatos durante la consulta aunque las bases de datos vectoriales han sido uno de los puntos clave en la evolución de estos sistemas.

Empleamos bases de datos vectoriales ya que las LLM nos ofrecen la opción de trabajar con los **embeddings** que sabemos aportan un vector denso de características semánticamente relevante. Por lo tanto, podemos realizar una búsqueda por vectores proximos entendiendo que los textos que representan esos vectores serán clave para resolver la consulta en cuestión.

#### Vector search

Sin desplegar grandes soluciones, [FAISS](https://faiss.ai/) nos ofrece un medio sencillo por el que probar esta interacción. Elegiremos primeramente una fuente de la que leer la información.

In [3]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://python.langchain.com/docs/introduction/")
documents = loader.load()

También deberemos elegir un modelo capaz de construir los embeddings dado un texto. Este mismo modelo será consultado ante una pregunta para obtener los textos o mejor dicho, vectores, que ofrecen la información relevante para nuestra consulta.

In [5]:
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectorstore = FAISS.from_documents(documents, embedding_model)

Una vez estas piezas están dispuestas solo tenemos que invocar a la cadena de acciones necesarias. Para simplificar esta operativa LangChain ya dispone de este mecanismo implementado de forma que es sencillo construir el grafo correspondiente.

In [6]:
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", 
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

# La cadena con consulta a bd
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

In [7]:
result = qa_chain.invoke({"query": "¿Qué sabes de LangChain?"})
print(result['result'])

LangChain es un framework diseñado para desarrollar aplicaciones impulsadas por grandes modelos de lenguaje (LLMs).

Aquí tienes un resumen de lo que sé sobre LangChain, basado en el texto proporcionado:

*   **Propósito Principal:** Simplifica cada etapa del ciclo de vida de las aplicaciones LLM:
    *   **Desarrollo:** Permite construir aplicaciones usando sus componentes de código abierto e integraciones con terceros. Utiliza LangGraph para construir agentes con estado, con soporte de streaming y "human-in-the-loop".
    *   **Producción:** Se usa LangSmith para inspeccionar, monitorear y evaluar las aplicaciones, lo que permite optimizar y desplegar con confianza.
    *   **Despliegue:** LangGraph Platform convierte las aplicaciones de LangGraph en APIs y asistentes listos para producción.

*   **Funcionalidad Clave:**
    *   Implementa una interfaz estándar para modelos de lenguaje grandes y tecnologías relacionadas, como modelos de embedding y almacenes vectoriales.
    *   Se i

Podemos ver los extractos de texto rescatados que han sido añadidos a la consulta.

In [8]:
print(result["source_documents"])

[Document(id='8714df47-042e-494d-b4da-223bddb50a74', metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='\n\n\n\n\nIntroduction | 🦜️🔗 LangChain\n\n\n\n\n\n\n\n\nSkip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answering system 

Y si hemos activado la trazabilidad, también podremos consultarlo en LangSmith.

![](../assets/images/rag_faiss.png)